In [1]:
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# added to be able to run in Google Colab
import sys
sys.path.append('/content/gdrive/MyDrive/UTRECHT/utils')
sys.path.insert(0,'/content/gdrive/MyDrive/UTRECHT')

sys.path.append('/content/gdrive/MyDrive/UTRECHT/utils_clsWB')

import utils
import utils_clsWB

In [3]:
from __future__ import print_function, division
import os, random, time, copy
from skimage import io, transform, morphology, feature
import numpy as np
import os.path as path
import scipy.io as sio
from scipy import misc
from scipy import ndimage, signal
import scipy
import pickle
import sys
import math
import matplotlib.pyplot as plt
import PIL.Image
from io import BytesIO
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler 
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

from utils.flow_functions import *
from utils.dataset_padding4cls import *
from utils.network_arch_cls import *
# from utils.trainval_cls_withMasking import *
from utils.trainval_cls_withMasking_WB import *

from utils_clsWB.eval_funcs import *
from utils_clsWB.dataset_CIFAR100LT import *
from utils_clsWB.network_arch_resnet import *
# from utils_clsWB.trainval import *
from utils_clsWB.plot_funcs import *
from utils_clsWB.regularizers import *
from utils_clsWB.class_balanced_loss import CB_loss

import warnings # ignore warnings
warnings.filterwarnings("ignore")
print(sys.version)
print(torch.__version__)

3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
2.0.0+cu118


#Setup config parameters
There are several things to setup, like which GPU to use, where to read images and save files, etc. Please read and understand this. By default, you should be able to run this script smoothly by changing nothing.


In [ ]:
# Here define the path, which is used to save the log and trained model in training process

project_name = 'clsPredMaskPadNoInd_resnet_ShuWeights_2022_06_15' 

# cpu or cuda
device ='cpu'
if torch.cuda.is_available(): 
    device='cuda:0'
print(device)

freqShow = 100

#model parameters
batch_size = 4
newSize = [512, 512]
total_epoch_num = 150    #300  # total number of epoch in training
base_lr = 0.0001      # base learning rate/


exp_dir = '/content/gdrive/MyDrive/UTRECHT/Class/exp' # experiment directory, used for reading the init model
save_dir = os.path.join(exp_dir, project_name) # where to save the log file and trained models.
if not os.path.exists(save_dir): 
    os.makedirs(save_dir)

cuda:0


In [ ]:
with open('/content/gdrive/MyDrive/UTRECHT/Class/segDistTransform_v2_2022_02_08_batch4_noNPP/dbinfo_kway_cls_stacks.plk', 'rb') as handle:
    dbinfo = pickle.load(handle)

cls_datasets = {set_name: PollenCls(dbinfo, size=newSize, set_name=set_name, maskType='pred')
                for set_name in ['train', 'test']}


dataloaders = {set_name: DataLoader(cls_datasets[set_name],                                    
                                    batch_size=batch_size,
                                    shuffle=set_name=='train', 
                                    num_workers=4) # num_work can be set to batch_size
               for set_name in ['train', 'test']}

print(len(cls_datasets['train']), len(cls_datasets['test']))

2474 559


In [ ]:
print(dbinfo.keys())

dict_keys(['perClassCount', 'train_cls_imgList', 'train_cls_predMaskList', 'train_cls_className', 'train_cls_classID', 'test_cls_imgList', 'test_cls_predMaskList', 'test_cls_className', 'test_cls_classID', 'species_codes_dict'])


In [ ]:
# for k-way classification, the dataset is true-positives and false-positives. 

In [ ]:
perClassCount = dbinfo['perClassCount']
train_cls_imgList = dbinfo['train_cls_imgList']
# dbinfo['train_cls_grndMaskList'] = train_cls_grndMaskList
train_cls_predMaskList = dbinfo['train_cls_predMaskList']
train_cls_className = dbinfo['train_cls_className']
train_kway_cls_classID = dbinfo['train_cls_classID']

test_cls_imgList = dbinfo['test_cls_imgList']
# dbinfo['test_cls_grndMaskList'] = test_cls_grndMaskList
test_cls_predMaskList = dbinfo['test_cls_predMaskList']
test_cls_className = dbinfo['test_cls_className']
test_kway_cls_classID  = dbinfo['test_cls_classID']

species_codes_dict = dbinfo['species_codes_dict']

In [ ]:
print(train_cls_imgList)

['/content/gdrive/MyDrive/UTRECHT/Class/segDistTransform_v2_2022_02_08_batch4_noNPP/Stacks/train/Images/lyc/PALIV_27-28_slide 1_scan1_35_1', '/content/gdrive/MyDrive/UTRECHT/Class/segDistTransform_v2_2022_02_08_batch4_noNPP/Stacks/train/Images/lyc/PALIV_27-28_slide 1_scan1_50_1', '/content/gdrive/MyDrive/UTRECHT/Class/segDistTransform_v2_2022_02_08_batch4_noNPP/Stacks/train/Images/lyc/PALIV_27-28_slide 1_scan1_50_2', '/content/gdrive/MyDrive/UTRECHT/Class/segDistTransform_v2_2022_02_08_batch4_noNPP/Stacks/train/Images/lyc/PALIV_27-28_slide 1_scan1_127_1', '/content/gdrive/MyDrive/UTRECHT/Class/segDistTransform_v2_2022_02_08_batch4_noNPP/Stacks/train/Images/lyc/PALIV_27-28_slide 1_scan1_185_1', '/content/gdrive/MyDrive/UTRECHT/Class/segDistTransform_v2_2022_02_08_batch4_noNPP/Stacks/train/Images/lyc/PALIV_27-28_slide 1_scan1_220_1', '/content/gdrive/MyDrive/UTRECHT/Class/segDistTransform_v2_2022_02_08_batch4_noNPP/Stacks/train/Images/lyc/PALIV_27-28_slide 1_scan1_261_1', '/content/gdriv

In [ ]:
class_selection = ['iso','poa', 'lyc', 'aln','hup', 'u/m', 'ast', 'cyp', 'hed', 'mol', 'pla', 'False_det' ]

In [ ]:
kway_train_cls_imgList = []
kway_test_cls_imgList = []
# kway_train_cls_grndMaskList = []
# kway_test_cls_grndMaskList = []
kway_train_cls_predMaskList = []
kway_test_cls_predMaskList = []
kway_train_cls_className = []
kway_test_cls_className = []
new_train_kway_cls_classID = []
new_test_kway_cls_classID = []

perClassCount = [0]*78


for set_name in ['train', 'test']:
    if set_name == 'train':
        for i in range(len(train_cls_imgList)):
            if train_cls_className[i] in class_selection:
                kway_train_cls_imgList.append(train_cls_imgList[i])
                # kway_train_cls_grndMaskList.append(train_cls_grndMaskList[i])
                kway_train_cls_predMaskList.append(train_cls_predMaskList[i])
                kway_train_cls_className.append(train_cls_className[i])
                new_train_kway_cls_classID.append(train_kway_cls_classID[i])
                perClassCount[train_kway_cls_classID[i]] += 1

    if set_name == 'test':
        for i in range(len(test_cls_imgList)):
            if test_cls_className[i] in class_selection:
                kway_test_cls_imgList.append(test_cls_imgList[i])
                # kway_test_cls_grndMaskList.append(test_cls_grndMaskList[i])
                kway_test_cls_predMaskList.append(test_cls_predMaskList[i])
                kway_test_cls_className.append(test_cls_className[i])
                new_test_kway_cls_classID.append(test_kway_cls_classID[i])
                perClassCount[test_kway_cls_classID[i]] += 1

print(len(kway_train_cls_imgList), len(kway_test_cls_imgList))
        # if the predicted label is 1, append img, grndMask, predMask, className, and k-way classID to respective lists.

2035 489


In [ ]:
dbinfo = {}
# dbinfo['meta'] = mapping
dbinfo['perClassCount'] = perClassCount
dbinfo['train_cls_imgList'] = kway_train_cls_imgList
# dbinfo['train_cls_grndMaskList'] = train_cls_grndMaskList
dbinfo['train_cls_predMaskList'] = kway_train_cls_predMaskList
dbinfo['train_cls_className'] = kway_train_cls_className
dbinfo['train_cls_classID'] = new_train_kway_cls_classID

dbinfo['test_cls_imgList'] = kway_test_cls_imgList
# dbinfo['test_cls_grndMaskList'] = test_cls_grndMaskList
dbinfo['test_cls_predMaskList'] = kway_test_cls_predMaskList
dbinfo['test_cls_className'] = kway_test_cls_className
dbinfo['test_cls_classID'] = new_test_kway_cls_classID 

dbinfo['species_codes_dict'] = species_codes_dict

with open('/content/gdrive/MyDrive/UTRECHT/Class/segDistTransform_v2_2022_02_08_batch4_noNPP/dbinfo_kway_cls_stacks_12class.plk', 'wb') as handle:
    pickle.dump(dbinfo, handle) # , protocol=pickle.HIGHEST_PROTOCOL

In [ ]:
with open('/content/gdrive/MyDrive/UTRECHT/Class/segDistTransform_v2_2022_02_08_batch4_noNPP/dbinfo_kway_cls_stacks_12class.plk', 'rb') as handle:
    dbinfo = pickle.load(handle)

dbinfo.keys(), len(dbinfo['train_cls_imgList']), len(dbinfo['test_cls_imgList'])

(dict_keys(['perClassCount', 'train_cls_imgList', 'train_cls_predMaskList', 'train_cls_className', 'train_cls_classID', 'test_cls_imgList', 'test_cls_predMaskList', 'test_cls_className', 'test_cls_classID', 'species_codes_dict']),
 2035,
 489)